<a href="https://colab.research.google.com/github/PmasCastro/Ufo_analysis/blob/main/Ufo_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Section




In [3]:
!pip install folium openpyxl
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install nbstripout
!nbstripout --install
#!nbstripout --attributes 'metadata.widgets'

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: bundlerextension console dejavu execute kernel
kernelgateway kernelspec migrate nbclassic nbconvert nbextension notebook run
server serverextension troubleshoot trust

Jupyter command `jupyter-labextension` not found.
Enabling notebook extension jupyter-js-widgets/extension

In [6]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display
import os

def load_data():
    # Try to detect if running in Google Colab
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        from google.colab import files
        print("Running in Colab. Please upload your Excel file:")
        uploaded = files.upload()
        if len(uploaded) == 0:
            print("❌ No file uploaded.")
            return None
        file_name = list(uploaded.keys())[0]
        df = pd.read_excel(file_name)
    else:
        file_name = "ufo_sightings_scrubbed.xlsx"
        if not os.path.exists(file_name):
            print(f"❌ Error: File '{file_name}' not found in {os.getcwd()}")
            return None
        df = pd.read_excel(file_name)

    df.columns = df.columns.str.strip().str.lower()
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df = df.dropna(subset=['latitude', 'longitude'])
    print(f"✅ Data loaded and cleaned. Total records: {len(df)}")
    return df


df = load_data()

if df is not None and not df.empty:
    country_options = ['All'] + sorted(df['country'].dropna().unique().tolist()) if 'country' in df.columns else ['All']
    shape_options = ['All'] + sorted(df['shape'].dropna().unique().tolist()) if 'shape' in df.columns else ['All']

    country_dropdown = widgets.Dropdown(options=country_options, description='Country:')
    shape_dropdown = widgets.Dropdown(options=shape_options, description='Shape:')
    ui = widgets.VBox([country_dropdown, shape_dropdown])

    def show_map(country='All', shape='All'):
        filtered_df = df
        if country != 'All':
            filtered_df = filtered_df[filtered_df['country'] == country]
        if shape != 'All':
            filtered_df = filtered_df[filtered_df['shape'] == shape]

        if filtered_df.empty:
            print("No data matches the selected filters.")
            return

        filtered_df = filtered_df.head(1000)

        m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
        cluster = MarkerCluster().add_to(m)

        for _, row in filtered_df.iterrows():
            popup = (
                f"{row.get('datetime', 'N/A')}<br>"
                f"{row.get('city', 'N/A')}, {row.get('state', 'N/A')}<br>"
                f"Shape: {row.get('shape', 'N/A')}<br>"
                f"{row.get('comments', 'N/A')}"
            )
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=popup,
                icon=folium.Icon(color="purple", icon="info-sign")
            ).add_to(cluster)

        display(m)

    interactive_out = widgets.interactive_output(show_map, {'country': country_dropdown, 'shape': shape_dropdown})
    display(ui, interactive_out)

else:
    print("❌ Map not created. Please check your data file and try again.")

Running in Colab. Please upload your Excel file:


Saving ufo_sightings.xlsx to ufo_sightings.xlsx
✅ Data loaded and cleaned. Total records: 0
❌ Map not created. Please check your data file and try again.
